# Arrow detection using tensorflow

## 1. Create folder hierarchy

In [ ]:
# autoreload & essential imports
%load_ext autoreload
%autoreload 2

import os

In [ ]:
# setup global and directory structure
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_ssd' 
# PRETRAINED_MODEL_NAME = 'ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_ssd_resnet152' 
# PRETRAINED_MODEL_NAME = 'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_ssd_resnet50' 
# PRETRAINED_MODEL_NAME = 'ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'

# CUSTOM_MODEL_NAME = 'my_centernet_mobilenetv2fpn' 
# PRETRAINED_MODEL_NAME = 'centernet_mobilenetv2_fpn_kpts'
# PRETRAINED_MODEL_URL = 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1?tf-hub-format=compressed'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),

    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': TF_RECORD_SCRIPT_NAME, 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

# create workspace directories
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}

## 2. Download pretrained model

In [ ]:
# download && unzip 
!wget --no-proxy {PRETRAINED_MODEL_URL} -O {PRETRAINED_MODEL_NAME}.tar.gz
!mv {PRETRAINED_MODEL_NAME + '.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

## 3. Clone tensorflow and build models API

In [ ]:
!git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Build API buffers && Install TensorFlow Object Detection
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=.
!cd Tensorflow/models/research && cp object_detection/packages/tf2/setup.py . && python3 -m pip install .

In [ ]:
# Verify Installation
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python3 {VERIFICATION_SCRIPT}
import object_detection

## 4. Create Label Map

In [ ]:
labels = [{'name':'licence', 'id':1}]
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
import glob

DATA='/home/ox330/data/licenseplate/'

annotation_jsons = glob.glob(DATA + '*.xml')
test_size = int(len(annotation_jsons) * 0.1)
test = annotation_jsons[:test_size]
train = annotation_jsons[test_size:]
print("Test size: %d" % len(test))
print("Train size: %d" % len(train))

!mkdir -p {os.path.join(paths['IMAGE_PATH'], 'test')}
!mkdir -p {os.path.join(paths['IMAGE_PATH'], 'train')}

!rm {os.path.join(paths['IMAGE_PATH'], 'test')}/*
!rm {os.path.join(paths['IMAGE_PATH'], 'train')}/*

for test1 in test:
    !cp {test1} {os.path.join(paths['IMAGE_PATH'], 'test')}
    !cp {os.path.splitext(test1)[0]+'.png'} {os.path.join(paths['IMAGE_PATH'], 'test')}
for train1 in train:
    !cp {train1} {os.path.join(paths['IMAGE_PATH'], 'train')}
    !cp {os.path.splitext(train1)[0]+'.png'} {os.path.join(paths['IMAGE_PATH'], 'train')}

## 5. Create TF records

In [ ]:
import generate_tfrecord as gen;
gen.xml_to_tfrecord(os.path.join(paths['IMAGE_PATH'], 'train'), files['LABELMAP'],
    os.path.join(paths['IMAGE_PATH'], 'train'),
    os.path.join(paths['ANNOTATION_PATH'], 'train.record'))
gen.xml_to_tfrecord(os.path.join(paths['IMAGE_PATH'], 'test'), files['LABELMAP'],
    os.path.join(paths['IMAGE_PATH'], 'test'),
    os.path.join(paths['ANNOTATION_PATH'], 'test.record'))

## 6. Copy Model Config to Training Folder

In [ ]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} \
    {os.path.join(paths['CHECKPOINT_PATH'])}

## 7. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
# config from pretrained model
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    text_format.Merge(f.read(), pipeline_config)

In [ ]:
# update model training parameters
pipeline_config.model.ssd.num_classes = len(labels)
# pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 512
# pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 512

pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'],
                                                                 PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'],
                                                                                        'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'],
                                                                                          'test.record')]

In [ ]:
#store updated config
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

## 8. Train the Model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(
    TRAINING_SCRIPT,
    paths['CHECKPOINT_PATH'],
    files['PIPELINE_CONFIG'])

In [ ]:
!rm Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/ckpt-*
!rm Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/checkpoint

In [ ]:
print(command)

In [ ]:
!{command}

## 9. Evaluate the Model

In [ ]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(
    TRAINING_SCRIPT,
    paths['CHECKPOINT_PATH'],
    files['PIPELINE_CONFIG'],
    paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

## 10. Detect from an Image

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt

import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'Cars274.png')

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
detections